File này để chuẩn bị dữ liệu, chia train là 2020, test là 2021

In [1]:
import pandas as pd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

rs = 12

In [2]:
df = pd.read_excel('../data/Data3.xlsx')
df['DateKey'] = df['DateKey'].map(lambda x: f'{str(x)[:4]}-{str(x)[4:6]}-{str(x)[6:]}')
df['DateKey'] = pd.to_datetime(df['DateKey'])
df

,DateKey,User number,Full name,Product Name,Product Type,Quantity,Regular price
0,2021-05-19,104002313,Nguyễn Phượng,Membership (2M_CN005_BUY2GIVE1),Membership,1,798000
1,2021-05-18,107000624,Phong Quân,Membership (6M_CN007_PO_2),Membership,1,1800000
2,2021-05-18,107000625,Nguyễn Hậu,Membership (6M_CN007_PO_2),Membership,1,1800000
3,2021-05-18,107000626,Nguyễn Ân,Membership (6M_CN007_PO_2),Membership,1,1800000
4,2021-05-17,105004342,Trần Ngân,Membership (2M_CN005_BUY2GIVE1),Membership,1,798000
...,...,...,...,...,...,...,...
29989,2020-08-23,105000722,Trần Thư,Membership (6M_CN001_GENERAL),Membership,1,2300000
29990,2020-08-23,105000723,Tăng Tùng Hòa,Membership (1M_CN005_PO_2),Membership,1,319000
29991,2020-08-23,105000724,Trần Trâm,Membership (1M_CN005_PO_2),Membership,1,319000
29992,2020-08-23,105000726,Mai Tai,Membership (1M_CN005_PO_2),Membership,1,319000


In [3]:
# chỉ lấy 2020
df = df[df['DateKey'].dt.year == 2020]

In [4]:
df.to_csv('../data/data.csv', index=False)

In [5]:
df.shape

(19784, 7)

# Xử lý dữ liệu

In [6]:
%run ../processing_data.py ../data/data.csv ../data/data_processed.csv

# Tạo RFM data

In [7]:
data_processed = pd.read_csv('../data/data_processed.csv', parse_dates=[0])

In [8]:
recency_unit = 1
snapshot_date = dt.datetime(year=2022, month=12, day=31)

def get_recency(d):
    return (snapshot_date - d.max()).days // recency_unit

# recency: lấy ngày snapshot - ngày mua cuối
# frequency: tổng quantity
# monetary: average số tiền mua

scaler = MinMaxScaler()

def get_rfm_data(dataframe):
    dataframe = (dataframe
            .groupby(['User number'])
            .agg({'DateKey': get_recency, 'Quantity': 'count', 'Regular price': 'sum'})
    )
    user_id = dataframe.index
    rfm = dataframe.values
    rfm = scaler.fit_transform(rfm) # normalize
    return pd.DataFrame(rfm, index=user_id, columns=['Recency', 'Frequency', 'Moneytary'])

In [9]:
rfm_data = get_rfm_data(data_processed)

In [10]:
rfm_data.to_csv('../data/rfm_data.csv')

In [11]:
rfm_data.shape

(8943, 3)